# Transparency in Tracking Maji Ndogo's Water Funds

### Introduction

As part of the Maji Ndogo water source improvement project, we received a comprehensive report highlighting key areas for strategic action. This data has not only informed our budget planning but also facilitated conversations with international funders who are supporting solutions to address this critical issue.

The journey to improve water access has officially begun, and we’re thrilled to share that the first few water sources are now operational. But there’s one more crucial task at hand—building a dashboard to showcase the impact of this initiative.

This dashboard will serve two important purposes:

1. **Public Transparency**: To show stakeholders and the community where the funds are being allocated and the tangible outcomes of their support.

2. **Project Oversight**: To empower decision-makers with actionable insights, ensuring the project stays on track and resources are utilized effectively.

Let’s dive in and explore how Power BI can help visualize this journey.

### Optimizing the Power BI Dashboard for Better Performance

If the Power BI file is becoming slow, these steps can help optimize the .pbix file for smoother operation:

1. Remove the following columns:
    - `address`, `province`, and `comments` columns from `project_progress` table
    - `date`, `biological`, `description`, and `pollutant_ppm` columns from `well_pollution`
    - `address` column from `location`

2. Remove the `queue_composition` table and `water_source_related_crime tables`.

### Updating the Dataset 

Next up, we’ll update our dataset to include new data. If we replace the Excel file we have, it will load the new data in our model. But, remember, this might break our report from last time, so to avoid that, we will store the Excel file somewhere else, and then edit the Data source settings in the power query editor to link to the new location.

After loading the updated dataset, ensure the model loads without errors. Once verified, look at `project_progress` table to observe the following changes:

- The province column has been removed.

- The town column now includes town names, but now Amina and Harare have the first letter of the province at the end so we know which Harare we're talking about and rural areas are also now appended with the name of the province! This is updated in the location too

This update will break the `budgeted_improvement_cost` column in the `project_progress` table we calculated last time. Below is the DAX to fix it. Since rural areas now have a province name attached, we need to search for a string rather than having to match it exactly:

In [ ]:
'''budgeted_improvement_cost =
    IF(
        CONTAINSSTRING(
            'project_progress'[town],
            "Rural*"
        ),
        RELATED('infrastructure_cost'[Rural_adjusted_cost]), RELATED('infrastructure_cost'[unit_cost_USD])
)'''

Here are the new columns introduced to the dataset:

- `date_started`: Captures the date when an improvement project began.
- `date_of_completion`: Indicates when the project was completed.
- `cost`: Represents the actual cost of the project in USD. This cost is the amount we paid to make the upgrade including travel expenses, materials, and labor costs. In an ideal world, this value would align with the budget outlined in the `budgeted_improvement_cost` column we created earlier to budget for the project.
- `assigned_vendor`: Stores the unique ID of the vendor that made the upgrade.

Finally, check the relationships between tables in the data model. For example, verify the relationship between `water_source` and `project_progress` to ensure it aligns with the project logic. Relationships and their directionalities may change during the import process, so ensure they are correct and consistent with the intended data flow.

Next up is the vendor_IDs and while they are functional, they are a little impersonal. If we import the Excel file again, we’ll find a `vendors` table. We won't use it, but here you can see who the heroes are on the ground, improving the lives of our people! We will import the `vendors` table into the data model ensuring the data is loaded correctly, so we have access to key details like the company name, who
owns it, and what they do:

- Groundwater Extraction teams drill wells.
- Water Distribution System Installation teams install taps.
- Water Purification System Installation teams install the UV and RO well filters.
- Civil Infrastructure Assessment teams are working on the broken infrastructure.

Let's check the data in `project_progress`. For each project, we now know if it is **Complete**, or still in the **Backlog**, and once it is complete, we can see which team worked on it, when the team started working on the project, when they completed it, and how much we paid them for the
improvements. Additionally, we have all of the other data we had before, giving us a rich dataset to work with. With this, we’re set to create some insightful visualizations!

### Planning the Dashboard

Before building the dashboard, it’s important to consider the stories we want it to tell. Our obJective is the dashboard has to make the work we're
doing visible to the public and help decision-makers monitor the progress of the project.

#### Public Perspective
As a member of the public, I want access to clear and simple insights about the project's progress, including:

1. The current status of the project: Which sources have been completed?
2. How much money has been spent so far?
3. A breakdown of where the money has been spent.
4. What the money was spent on?
5. Details about everything in my town.

#### Decision-Maker Perspective
As a decision-maker, I require detailed and actionable data to oversee the project's progress effectively, including:

1. A summary of the project's current stage and progress.
2. The total amount of money spent so far.
3. A breakdown of where the money has been spent.
4. What the money was spent on?
5. An analysis of whether the current budget will be sufficient to complete the project.
6. Where can we cut costs?
7. Data visualization and analysis at the national, provincial, and town levels for strategic planning.

we'll create a unified dashboard that serves both purposes, as these user stories are quite similar. We'll incorporate detailed drill-throughs to access more specific content when necessary.

First, let's design a basic layout for the dashboard, then dive into what we need to calculate.

![National Dashboard](visualisation/part8/assets/national_dashboard.png)

So the main visuals we want to build are:

- A map that will show us how far along the project is. Users should be able to see visually how far the project is. Users should also be able to select any town, or rural area, and know immediately how the project is doing.
- A cost tracking visual. A KPI plot is a nice option that will show us how much we thought it would cost, vs. how much it is really costing us.
- Break down the cost of the project so people can see where the money was spent.
- Summaries of key metrics:
    • Project progress (%)
    • Basic access to water (%), updated now that the new water sources are installed.
    • How many people are affected by the changes.? Remember, we're doing this for the citizens of Maji Ndogo!
    • How far to go before the project is complete?

Each visual will be interactive at the national, provincial, and local levels. Additionally, they will be designed to allow users to analyze trends over time, enabling a comprehensive overview of the project's journey.

### Creating the Enhanced Map 

To build the new and improved map, we will use the shape map JSON file: `MD_Full_map.json` which will serve as the map visual in our project.

In this case, we will use the `town` column from the `project_progress` table in the Location well. By adding the `source_status` field to the Color Saturation well, the map will show us how many water sources exist in each location, though it won't indicate their status or completion rates. To make the visual more meaningful, we need to create a measure to reflect the `Project Progress (%)` metric.

The challenge lies in the need for the measures to interact correctly with the location and date filters. While the calculations themselves are straightforward, the filters introduce additional complexity.

To calculate the Project progress (%), we need to calculate the number of complete projects divided by all of the projects. At a national level, it
includes all of the projects in Maji Ndogo. However, for specific towns like Djenne, it will be the number of projects completed in Djenne, divided by the number of projects in Djenne in total. So we need the measure to always take into account the town filter.

### Total Improvements

To calculate the total number of improvements, we use the following DAX expression:

In [ ]:
'''total_improvements = 
    CALCULATE(
        COUNTROWS('project_progress'),
        ALLEXCEPT('project_progress', 'project_progress'[town])
    )'''

Let’s break it down:

1. **CALCULATE**: This function lets us modify how we calculate a measure based on specific conditions, which in this case are set by two conditions:
- ALLEXCEPT(`project_progress`, `project_progress[town]`)
- ALL(`project_progress[date_of_completion]`)
2. **COUNTROWS('project_progress')**: This part simply counts the number of rows in the `project_progress` table, which gives us the total count of improvements.
3. **ALLEXCEPT('project_progress', 'project_progress'[town])**: This condition ensures that Power BI ignores all filters except the ones in the town column. So, no matter what other filters are applied elsewhere, the only one that will apply is the town column.

> **NOTE**: Adding a date slicer as a filter will create problems. When we change the date, the rows are filtered by the date, which means we don't get the total number of projects. What we get is the total number of projects in the `project_progress` table, but if we select a town, it calculates all of the improvements in that town.

### Completed Projects

What is the most reliable way to know whether a project is complete? Well, we can count the number of rows that satisfy the condition for a project being complete. We should get a number of 29.

In [ ]:
'''number_completed_projects = 
    CALCULATE(
        COUNTROWS('project_progress'),
        'project_progress'[date_of_completion] <> BLANK()
    )'''

Now that we have that, we'll go ahead and calculate the percentage (`pct_project_complete`) and format it so it will show up as a % with no
decimals.

In [ ]:
'''pct_project_complete = 
    DIVIDE(
        'project_progress'[number_completed_projects],
        'project_progress'[total_improvements]
    )'''

Our value should be 0% at the start on a national level, but Serowe, for example, is 1% done. We will add the values to the dashboard as cards.

Additionally, we included a new metric, `More Sources to Go` calculated using the formula below:

In [ ]:
'''more_sources_to_go = 
    CALCULATE(
        'project_progress'[total_improvements] - 
        'project_progress'[number_completed_projects]
    )'''

Now we also have a good metric to show progress on the map, we will use `pct_project_complete` as the colour saturation variable. Since we used percentages, the visual will scale better, so the colours are now more informative than before. 

Below is the updated map visualisation ↓ 

![Project Progress Map](visualisation/part8/assets/progress_map.png)

Towns like Zuri, Mrembo, and Isiqalo are displayed in grey to indicate that no projects have been undertaken in those areas. The colored towns represent the progress of completed projects, with the saturation level indicating the number of projects completed— more saturated towns have a higher number of completed projects.

### Basic Access to Water

Next up, we calculate the Basic Access to Water statistics. While we performed a similar calculation last week, this time we include the newly added water sources. Rivers have been replaced by wells, and once the wells are completed, they are considered clean. As a result, citizens with access to these wells now have basic access to water from an improved source. The formula for Basic Access to Water is as follows:

In [ ]:
'''total_population = 
    CALCULATE(
        SUM('water_source'[number_of_people_served]),
        ALLEXCEPT(
            'project_progress', 
            'project_progress'[town]
        )
    )'''

The population of Maji Ndogo should be 28 million, while Kilimani's population should be 4 million. 

Now we will calculate `population_with_basic_access`. Previously, we used a column to classify sources as Basic or not and then added the rows. However, since the dashboard performance is slowing down, we will now leverage DAX to create a measure that performs the calculation in one step. This approach will prevent Power BI from storing all those strings in the table, resulting in faster calculations and reduced memory usage. Here we basically calculate the sum of people who use sources that fulfill the conditions of basic water.

In [ ]:
'''population_with_basic_access = 
    CALCULATE( 
        SUM('water_source'[number_of_people_served]),
        FILTER(
            ALL(water_source),
            OR( 
                OR(
                    AND( 
                        'water_source'[type_of_water_source] = "well",
                        RELATED(well_pollution[results]) = "Clean"
                    ),
                    'water_source'[type_of_water_source] = "tap_in_home"
                ),
                AND( 
                    'water_source'[type_of_water_source] = "shared_tap",
                    'water_source'[Average_queue_time] < 30
                )
            )
        )
    )'''

### Access to Basic Water After Improvements

We will calculate the number of people who will have access to basic water once the water source is improved as `population_now_basic_access`. The expected result is that **10,760** people will gain access to basic water after the improvement (excluding everyone who already had basic access).

In [ ]:
'''population_now_basic_access = 
    CALCULATE(
        SUM('water_source'[number_of_people_served]), 
        'project_progress'[source_status] = "Complete"
    )'''

Next, we'll calculate the number of people who will have access to basic water once all improvements are complete (including those who already had access) as `pct_population_now_basic_access`, and the result should be formatted as a percentage. It should be 34% at the start, and if
you select the town of Serowe in Hawassa, we should see 57% access.

In [ ]:
'''pct_population_now_basic_access = 
    DIVIDE(
        [population_with_basic_access] + [population_improved],
        [total_population]
    )'''

### Tracking Costs and Budget

Now, let’s dive into tracking costs! The key aspect we need to focus on is how much we have spent so far and whether we have exceeded our budget. Our budget from last time is our target, and ideally, we want to spend less than the budget​. To visualise this effectively, we can use the KPI visual, which will allow us to monitor our progress over time.

To make the visual functional, we’ll need to incorporate a date slicer, which will enable us to filter data based on selected date ranges.

For this visual, we need to calculate two measures; the running cost (`cumulative_cost`) and running budget (`cumulative_budget`). These
numbers should sum up all of the budgeted costs and real costs, up to a date, respectively. For example, if we paid vendors 100 000 USD on day one
and 150 000 USD on day two, the running cost is 250 000 USD on day two.

Our calculation for `cumulative_budget` should sum up all of the budget costs for projects with completion dates <= the current date. Keep in mind that blanks in the data can cause issues during comparison, so we will need to handle them by removing them using DAX. With the current data, the result should be **$128,450**.

In [ ]:
'''cumulative_budget = 
    CALCULATE(
        SUM('project_progress'[budgeted_improvement_cost]),
        FILTER(
            ALL('project_progress'[date_of_completion]),
            'project_progress'[date_of_completion] <= MAX('project_progress'[date_of_completion]) &&
            NOT(ISBLANK('project_progress'[date_of_completion]))
        )
    )'''

To calculate `cumulative_cost`, we can simply copy-paste the budget and change the columns to reference the correct data. This should result in a total of **$131,914.91**.

To set it up, create a KPI visual, and drag the `cumulative_cost` into the Value well, the `cumulative_budget` into the Target well, and `date_of_completion` into the Trend axis well. Once these are set up, you should see a visual like this:

![KPI Visual](visualisation/part8/assets/KPI_visual.png)

The current format is not quite correct, so we need to adjust a few settings. First, we'll notice that the goal is 98.70K USD, while the actual expenditure is 131.91K USD. This means we have exceeded the budget, but the KPI is incorrectly interpreting this as positive with a correct tick mark. To fix this, change the Trend axis to "Low is Good." Additionally, update the Target label to "Budget."

Once these changes are made, the visual communicates the following:

- Trend of the overall cost over time;
- How much money we have spent;
- How much we wanted to spend; and
- With colours and icons, clearly show whether we have over-spent (Budget deficit) or if we spent less (Budget surplus).

We will add metrics like **People helped** (`population_now_basic_access`), **number of sources to still improve** (which is calculated as `total_improvements` minus `number_of_completed_projects`), and use the `Aggregated_improvements` column from last project to break down costs per improvement.

Now we will update our dashboard by incorporating all the visuals and slicers we've created.

![National Report](visualisation/part8/assets/national_report.png)

### Recap: Did We Achieve the User Stories?

Let's take a moment to review the user stories we set out to address:

1. How far have we come in the project? ✔
2. How much money has been spent so far? ✔
3. Where was the money been spent? ✔
4. What was the money spent on? ✔
5. Will we have enough funds to complete the project? ✔
6. Where can we cut costs? ✘
7. Can we view data at the national, provincial, and town levels? ✔

It's ok that we were not able to complete User Story 6 regarding cost-cutting at this stage. Since we lack pre-project data and we don't know where we can cut costs before the project starts, it's best to wait for incoming data to assist decision-makers in identifying cost-saving opportunities.

Once everything is polished, we'll be ready to share our dashboard with the world!

## 4 January 2024 Update: Welcome Back!

Hello again! After a year of improvements, we’re excited to share some updates to our data. This time, we’ll see the power of dashboards in action. Let’s dive in! 🚀

We have updated data (**up to 2024-01-03**) called `Md_water_services_data.xlsx`. This file contains the latest upgrades and changes to water service sources across Maji Ndogo. We will overwrite the previous Excel file with this new dataset or link as the updated source to keep everything in sync.

Let’s explore the improvements together! 🌊

![2023 National Report](visualisation/part8/assets/updated_report_2023.png)

Once everything is set up, we notice that the date slider has automatically expanded, allowing us to select data up to 2024! Without additional modeling or manually transferring results into spreadsheets, our analysis has seamlessly updated. If we were connected to a live SQL database, this update would have occurred instantly.

Let’s explore the latest insights. At the beginning of our project, water access stood at **34%**. Now, it’s at **48%**— a significant improvement! However, with the project only **22%** complete, there’s still a long journey ahead. At this pace, it’s estimated to take around five years to fully achieve our goals.

Now for the challenges. The budget KPI indicates we're currently **10%** over budget. Let’s dig deeper to understand the cause.

If we look at the different provinces, we can see that Sokoto has taken a large part of the budget, but we knew it would be expensive since we had
to drill many wells.

Focusing on Sokoto specifically, we can see it’s nearly **40%** over budget. The region's challenging conditions— limited road infrastructure and difficult terrain —have significantly increased operational costs. Unfortunately, this wasn’t adequately accounted for in the initial planning.

### Key Influencers and Cost Analysis

We are going to create a key indicators visual to analyse factors contributing to high costs considering variables such as town, province, location type (rural or urban), or the time taken to complete projects. 

![Key Influencers](visualisation/part8/assets/key_influencers.png)

Key insights from the Key Influencers analysis:

1. When we have to drill wells, it is expensive. We took this into account in our budget, so this is not the reason for the high costs.
2. When sources are in rural areas, costs are higher. We took this into account in our budget, so this is not the reason for the high costs.
3. When jobs take long, costs go up. This can be due to: 
    - The complexity of the task (e.g., installing multiple taps takes more time than installing a single filter).
    - Travel time between remote locations, which can accumulate significantly over the course of the project, adding to expenses.

We will now create a new visuals page and make several charts examining the effect on the average cost of an improvement and try to show the following
results:
- It is almost twice as expensive to improve a source in a rural area, compared to an urban area.
- Sokoto has a very high average cost of improvement, both rurally and in urban areas.
- We are over budget in every province.
- We underestimated the cost of rural improvements in Sokoto

![Cost Analysis](visualisation/part8/assets/cost_analysis.png)

While this data is useful to know, we can't do anything about it. Rural areas need larger vehicles that use more fuel. Sources are far apart, and
there are many of them. But we can't change that. So let's focus on finding a change we can make to make sure we stay on budget.

### Vendor Analysis and Cost Optimization

Let's look at the vendors. If we see vendors that are asking too much, we stop using them. Simple! To start, we’ll analyze vendor costs by calculating the average cost per vendor for improvements and sorting the results from the most expensive to the least expensive. But we need to compare apples to apples. Teams that drill wells will naturally charge
more than teams installing filters, and we also saw that it costs more to improve sources in Sokoto and Kilimani, so that's another factor.

To ensure a fair comparison, we’ll create a slicer for a date, improvement type (`aggregated_improvements`), and rural/urban split and add our map to this page. This allows us to dynamically filter and compare vendor prices. For example, we can select "Drill well" improvements and directly compare vendors’ costs for the same type of work.

![Vendor Performance](visualisation/part8/assets/vendor_performance.png)

When selecting the most expensive vendor, **MBS60**, a fascinating trend emerges. This vendor is exclusively drilling wells across the rural regions of Sokoto— a challenging environment with harsh conditions. Their higher costs start to make sense when considering the difficulty of their operations. The four most expensive vendors all operate either in Sokoto or Kilimani, areas characterized by their logistical and environmental challenges.

In contrast, the cheapest drilling teams work in urban areas outside of Sokoto, where access is easier. This brings us to a critical question: Who is actually expensive, and who isn’t?

Here, the data tells us if we fired the expensive teams, it would
have been a mistake.

Let’s examine vendors specializing in filter installations, specifically Reverse Osmosis (RO) and RO + UV systems. Focus on two vendors:

- **ERI893**: Renowned as the best water purification installers in the whole of Maji Ndogo.
- **OW290**: A less reputable company known as Ouagadougou Waterworks.

![Vendor Costs](visualisation/part8/assets/vendor_costs.png)

The data highlights that **ERI893** is the most expensive filter installer in Maji Ndogo, charging nearly 50% more than the cheapest vendor. This raises an interesting question: How can **ERI893** still be considered the best?

**Entebbe RO Installers** worked in rural areas and finished the most projects. Still unsure about their efficiency? Let's examine their work compared to others in similar conditions. We will filter the table to only include the data for rural areas and the Sokoto province.

![Sokoto Vendors](visualisation/part8/assets/sokoto_vendors.png)

**Entebbe RO Installers** is the best team in Maji Ndogo! Look how cheap they are when we compare them to the other teams in the right context.

While this observation is compelling, it highlights a crucial lesson about working with complex, real-world data: things are not always as straightforward as they seem. We may believe we’ve found an answer, only to discover later that there’s more to uncover. 

So, why are **ERI893** so cheap?

When we take a look at how Entebbe RO Installers and Ouagadougou Waterworks  operate across Maji Ndogo, Entebbe RO Installers primarily stay within one area, focusing on projects close to their base. This strategy minimizes travel costs and allows them to take on new projects more quickly. While Ouagadougou Waterworks frequently moves between towns and rural areas, incurring higher travel expenses and delays between projects.

Entebbe RO Installers has made the most out of all the purification teams, not because they charged a lot, but because their choices mean they get
more projects done! So what can we do? Well, we can get our designers and animators to make a video, and create some digital infographics we can send to the teams, explaining how they can improve, and how to search for jobs closer to them on the system. If one team follows our advice, we make a small change, but hopefully all of them take note, and pick jobs that are more optimal. This could even help us manage our budget more effectively!

The Power of Data
Isn’t it incredible how much insight can be unlocked from data? It’s a reminder of the value of curiosity and determination in uncovering the truth hidden within the numbers.

We'll check in again once we have more data. Stay tuned for the next round of insights!

## Final Update (3 December 2027)

Hello! Can you believe it has been almost four years? Time has flown by— it feels like just yesterday!

#### Final Dataset
For this analysis, we are using an updated dataset (`Md_water_services_data.xlsx`) as the data source for our dashboard. 

![Final Report](visualisation/part8/assets/updated_report_2027.png)

We have completed our project which successfully analysed and visualised water access data for the fictitious country of Maji Ndogo. Through a combination of data analysis using MySQL and insightful visualizations with Power BI, we have shown that all water sources are now accessible across the country. The integration of various data sources and the creation of a comprehensive dashboard provided a clear overview of water service availability offering valuable insights that supported future planning and decision-making in Maji Ndogo. This project demonstrated the power of data-driven insights in addressing essential infrastructure challenges.